In [35]:
%pip install tensorflow-macos tensorflow-metal gymnasium pygame


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 30.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 34.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
  Attempting unins

In [37]:
import tensorflow as tf

# A basic neural network with 1 input and 3 outputs
inputs = tf.keras.Input(shape=(1,))
layer1 = tf.keras.layers.Dense(512, activation="relu")(inputs)
outputs = tf.keras.layers.Dense(3, activation="linear")(layer1)

nn_model = tf.keras.Model(inputs=inputs, outputs=outputs)
nn_model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,563 (10.01 KB)

 Trainable params: 2,563 (10.01 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
input = tf.constant([[3.0]])  # input must be a 2D tensor
output = nn_model(input)
print(output)


tf.Tensor([[ 0.04137211 -0.00828392 -0.13355584]], shape=(1, 3), dtype=float32)


In [39]:
import numpy as np

inputs = np.arange(0, 1.1, 0.1)
for val in inputs:
    inp = tf.constant([[val]])
    out = nn_model(inp)
    print(f"Input: {val}, Output: {out.numpy()}")


Input: 0.0, Output: [[0. 0. 0.]]
Input: 0.1, Output: [[ 0.00137907 -0.00027613 -0.00445186]]
Input: 0.2, Output: [[ 0.00275814 -0.00055226 -0.00890372]]
Input: 0.30000000000000004, Output: [[ 0.00413721 -0.00082839 -0.01335559]]
Input: 0.4, Output: [[ 0.00551628 -0.00110453 -0.01780745]]
Input: 0.5, Output: [[ 0.00689535 -0.00138066 -0.02225931]]
Input: 0.6000000000000001, Output: [[ 0.00827442 -0.00165678 -0.02671118]]
Input: 0.7000000000000001, Output: [[ 0.00965349 -0.00193292 -0.03116303]]
Input: 0.8, Output: [[ 0.01103256 -0.00220906 -0.03561489]]
Input: 0.9, Output: [[ 0.01241164 -0.00248518 -0.04006676]]
Input: 1.0, Output: [[ 0.01379071 -0.00276132 -0.04451863]]


In [ ]:
import gymnasium as gym

e = gym.make('MountainCarContinuous-v0', render_mode='human')
obs = e.reset()
print("State tuple:", obs)

# Network with 2 inputs (position, velocity) → 1 output (action)
inputs = tf.keras.Input(shape=(2,))
layer1 = tf.keras.layers.Dense(512, activation="relu")(inputs)
outputs = tf.keras.layers.Dense(1, activation="linear")(layer1)
nn_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Run agent
state, _ = e.reset()
done = False

while not done:
    input_tensor = tf.convert_to_tensor([state], dtype=tf.float32)
    action = nn_model(input_tensor).numpy()
    state, reward, done, truncated, info = e.step(action)
    e.render()


State tuple: (array([-0.4135012,  0.       ], dtype=float32), {})


In [1]:
import gymnasium as gym
import tensorflow as tf

# Create the environment
env_name = "BreakoutNoFrameskip-v4"
e = gym.make(env_name, render_mode='human')
state, _ = e.reset()

# Create CNN-style model
inputs = tf.keras.Input(shape=(210, 160, 3,))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(512, activation="relu")(x)
outputs = tf.keras.layers.Dense(4, activation="linear")(x)
nn_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Fire to start the game
state, reward, done, truncated, info = e.step(1)

# Run 100 steps
for i in range(100):
    state_tensor = tf.expand_dims(state, axis=0)
    action_values = nn_model(state_tensor)
    action = tf.argmax(action_values[0]).numpy()
    print("Action values:", action_values.numpy(), "Selected action:", action)
    state, reward, done, truncated, info = e.step(action)
    e.render()


: 